
# Points critiques

Code généré avec le support de Claude.ai.

## Objectif

Construire une fonction univariée continuellement différentiable (classe C¹) possédant :
1. Un **point selle** (ni minimiseur ni maximiseur)
2. Au moins un **minimiseur local** (non-global)
3. Au moins un **maximiseur local**
4. Un **minimiseur global**

## Stratégie Générale

La fonction est construite par **morceaux** en trois zones :
- **Zone 1** (x ≤ -1) : Polynômes d'Hermite cubiques
- **Zone 2** (-1 ≤ x ≤ 1) : Polynôme de degré 5 avec point selle
- **Zone 3** (x ≥ 1) : Polynômes d'Hermite cubiques

Cette approche hybride permet de :
- Garantir mathématiquement un point selle en x = 0
- Assurer la continuité C¹ aux jonctions x = -1 et x = 1
- Contrôler précisément la position des extrema

---

## 1. Polynômes d'Hermite Cubiques

### Principe

Un polynôme d'Hermite cubique entre deux points (x₀, y₀) et (x₁, y₁) est défini par :

```
H(x) = h₀₀(t)·y₀ + h₁₀(t)·dx·dy₀ + h₀₁(t)·y₁ + h₁₁(t)·dx·dy₁
```

où :
- t = (x - x₀)/(x₁ - x₀) ∈ [0,1] (paramètre normalisé)
- dx = x₁ - x₀ (largeur de l'intervalle)
- dy₀, dy₁ sont les dérivées spécifiées aux extrémités
- h₀₀, h₁₀, h₀₁, h₁₁ sont les fonctions de base d'Hermite

### Fonctions de Base

Les fonctions de base d'Hermite garantissent les conditions d'interpolation :

```
h₀₀(t) = 2t³ - 3t² + 1       → H(x₀) = y₀
h₁₀(t) = t³ - 2t² + t         → H'(x₀) = dy₀
h₀₁(t) = -2t³ + 3t²           → H(x₁) = y₁
h₁₁(t) = t³ - t²              → H'(x₁) = dy₁
```

### Propriété Clé : Continuité C¹

Entre deux segments consécutifs [xᵢ, xᵢ₊₁] et [xᵢ₊₁, xᵢ₊₂], si on impose que la dérivée à droite du premier segment égale la dérivée à gauche du second segment (dyᵢ₊₁), alors **la fonction globale est automatiquement C¹**.

### Implémentation

```julia
function hermite_cubic(x, x0, x1, y0, y1, dy0, dy1)
    t = (x - x0) / (x1 - x0)
    h00 = 2*t^3 - 3*t^2 + 1
    h10 = t^3 - 2*t^2 + t
    h01 = -2*t^3 + 3*t^2
    h11 = t^3 - t^2
    
    dx = x1 - x0
    return h00*y0 + h10*dx*dy0 + h01*y1 + h11*dx*dy1
end
```

---

## 2. Zone Centrale : Polynôme avec Point Selle

### Construction du Point Selle

Pour avoir un **point selle** en x = 0, il faut que :
- f'(0) = 0 (point critique)
- f''(0) = 0 (point d'inflexion)

On utilise un polynôme de la forme :

```
f(x) = ax⁵ + bx³ + d
```

Ses dérivées sont :
```
f'(x) = 5ax⁴ + 3bx²
f''(x) = 20ax³ + 6bx
```

En x = 0 :
```
f'(0) = 0  ✓  (automatiquement satisfait)
f''(0) = 0  ✓  (automatiquement satisfait)
```

### Raccordement C¹ aux Jonctions

Le polynôme doit se raccorder de manière C¹ avec les zones 1 et 3 aux points x = -1 et x = 1.

**Conditions à satisfaire :**
1. f(-1) = y_{zone1}(-1) (continuité de f)
2. f(1) = y_{zone3}(1) (continuité de f)
3. f'(-1) = dy_{zone1}(-1) (continuité de f')
4. f'(1) = dy_{zone3}(1) (continuité de f')

**Système d'équations :**

À partir de f(x) = ax⁵ + bx³ + d :

```
f(-1) = -a - b + d = y₁
f(1) = a + b + d = y₃
f'(-1) = 5a + 3b = dy₁
f'(1) = 5a + 3b = dy₃
```

Notez que f'(-1) = f'(1) par symétrie du polynôme. On prend donc la moyenne :

```
dy_moy = (dy₁ + dy₃)/2
```

**Résolution :**

Addition de f(-1) et f(1) :
```
2d = y₁ + y₃  →  d = (y₁ + y₃)/2
```

Soustraction :
```
2a + 2b = y₃ - y₁  →  a + b = (y₃ - y₁)/2
```

De la condition sur les dérivées :
```
5a + 3b = dy_moy
```

En combinant les deux dernières équations :
```
b = (y₃ - y₁)/2 - a
5a + 3((y₃ - y₁)/2 - a) = dy_moy
5a + 3(y₃ - y₁)/2 - 3a = dy_moy
2a = dy_moy - 3(y₃ - y₁)/2

→ a = (dy_moy - 3(y₃ - y₁)/2)/2
```

Puis :
```
b = (y₃ - y₁)/2 - a
```

### Code

```julia
a_poly = (avg_dy - 3*(y_at_plus1 - y_at_minus1)/2) / 2
b_poly = (y_at_plus1 - y_at_minus1)/2 - a_poly
d_poly = (y_at_minus1 + y_at_plus1) / 2
```

In [ ]:
using Plots

# Construction manuelle avec polynômes d'Hermite cubiques pour garantir C¹
# Zone 1 (x ≤ -1) : polynômes d'Hermite par morceaux
# Zone 2 (-1 ≤ x ≤ 1) : polynôme avec point selle en x=0
# Zone 3 (x ≥ 1) : polynômes d'Hermite par morceaux

# Fonction pour créer un polynôme d'Hermite cubique entre deux points
function hermite_cubic(x, x0, x1, y0, y1, dy0, dy1)
    # Polynôme d'Hermite cubique: garantit C¹ entre (x0,y0,dy0) et (x1,y1,dy1)
    t = (x - x0) / (x1 - x0)
    h00 = 2*t^3 - 3*t^2 + 1
    h10 = t^3 - 2*t^2 + t
    h01 = -2*t^3 + 3*t^2
    h11 = t^3 - t^2
    
    dx = x1 - x0
    return h00*y0 + h10*dx*dy0 + h01*y1 + h11*dx*dy1
end

function hermite_cubic_derivative(x, x0, x1, y0, y1, dy0, dy1)
    t = (x - x0) / (x1 - x0)
    dt_dx = 1 / (x1 - x0)
    
    dh00 = (6*t^2 - 6*t) * dt_dx
    dh10 = (3*t^2 - 4*t + 1) * dt_dx
    dh01 = (-6*t^2 + 6*t) * dt_dx
    dh11 = (3*t^2 - 2*t) * dt_dx
    
    dx = x1 - x0
    return dh00*y0 + dh10*dx*dy0 + dh01*y1 + dh11*dx*dy1
end

# Points et dérivées pour la zone 1 (x ≤ -1)
x1_points = [-4.0, -3.0, -2.0, -1.0]
y1_points = [-1.0, -2.5, -3.0, -2.0]
dy1_points = [-1.5, -0.8, 0.0, 1.5]  # Dérivées (min local en x=-2 avec dy=0)

# Points et dérivées pour la zone 3 (x ≥ 1)
x3_points = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
y3_points = [2.0, 3.0, 1.5, -1.0, -4.5, -3.0, -1.5]
dy3_points = [1.5, 0.5, -2.0, -3.0, 0.0, 2.5, 1.5]  # Min global en x=5 (dy=0)

# Fonctions pour les zones 1 et 3
function f_zone1(x)
    for i in 1:length(x1_points)-1
        if x1_points[i] <= x <= x1_points[i+1]
            return hermite_cubic(x, x1_points[i], x1_points[i+1], 
                               y1_points[i], y1_points[i+1],
                               dy1_points[i], dy1_points[i+1])
        end
    end
    # Extrapolation si nécessaire
    i = length(x1_points) - 1
    return hermite_cubic(x, x1_points[i], x1_points[i+1], 
                       y1_points[i], y1_points[i+1],
                       dy1_points[i], dy1_points[i+1])
end

function f_zone1_prime(x)
    for i in 1:length(x1_points)-1
        if x1_points[i] <= x <= x1_points[i+1]
            return hermite_cubic_derivative(x, x1_points[i], x1_points[i+1], 
                                          y1_points[i], y1_points[i+1],
                                          dy1_points[i], dy1_points[i+1])
        end
    end
    i = length(x1_points) - 1
    return hermite_cubic_derivative(x, x1_points[i], x1_points[i+1], 
                                  y1_points[i], y1_points[i+1],
                                  dy1_points[i], dy1_points[i+1])
end

function f_zone3(x)
    for i in 1:length(x3_points)-1
        if x3_points[i] <= x <= x3_points[i+1]
            return hermite_cubic(x, x3_points[i], x3_points[i+1], 
                               y3_points[i], y3_points[i+1],
                               dy3_points[i], dy3_points[i+1])
        end
    end
    i = length(x3_points) - 1
    return hermite_cubic(x, x3_points[i], x3_points[i+1], 
                       y3_points[i], y3_points[i+1],
                       dy3_points[i], dy3_points[i+1])
end

function f_zone3_prime(x)
    for i in 1:length(x3_points)-1
        if x3_points[i] <= x <= x3_points[i+1]
            return hermite_cubic_derivative(x, x3_points[i], x3_points[i+1], 
                                          y3_points[i], y3_points[i+1],
                                          dy3_points[i], dy3_points[i+1])
        end
    end
    i = length(x3_points) - 1
    return hermite_cubic_derivative(x, x3_points[i], x3_points[i+1], 
                                  y3_points[i], y3_points[i+1],
                                  dy3_points[i], dy3_points[i+1])
end

# Construire le polynôme central avec les bonnes conditions de raccordement
y_at_minus1 = y1_points[end]
dy_at_minus1 = dy1_points[end]
y_at_plus1 = y3_points[1]
dy_at_plus1 = dy3_points[1]

# Polynôme f(x) = ax^5 + bx^3 + d
# f'(x) = 5ax^4 + 3bx^2
# f'(0) = 0 ✓, f''(0) = 0 ✓

# Conditions:
# f(-1) = -a - b + d = y_at_minus1
# f(1) = a + b + d = y_at_plus1
# f'(-1) = 5a + 3b = dy_at_minus1
# f'(1) = 5a + 3b = dy_at_plus1

# Note: f'(1) = f'(-1) par symétrie du polynôme
# Prenons la moyenne pour un meilleur raccordement
avg_dy = (dy_at_minus1 + dy_at_plus1) / 2

# Système:
# 5a + 3b = avg_dy ... (1)
# a + b = (y_at_plus1 - y_at_minus1)/2 ... (2)
# d = (y_at_minus1 + y_at_plus1)/2 ... (3)

# De (2): b = (y_at_plus1 - y_at_minus1)/2 - a
# Dans (1): 5a + 3((y_at_plus1 - y_at_minus1)/2 - a) = avg_dy
#           2a = avg_dy - 3(y_at_plus1 - y_at_minus1)/2

a_poly = (avg_dy - 3*(y_at_plus1 - y_at_minus1)/2) / 2
b_poly = (y_at_plus1 - y_at_minus1)/2 - a_poly
d_poly = (y_at_minus1 + y_at_plus1) / 2

println("Fonction Construite avec Polynômes d'Hermite Cubiques")
println("="^70)
println("\nCoefficients du Polynôme Central (zone [-1, 1]):")
println("  f(x) = $(round(a_poly, digits=5))x⁵ + $(round(b_poly, digits=5))x³ + $(round(d_poly, digits=5))")
println()

function f_poly(x)
    return a_poly * x^5 + b_poly * x^3 + d_poly
end

function f_poly_prime(x)
    return 5 * a_poly * x^4 + 3 * b_poly * x^2
end

function f_poly_double_prime(x)
    return 20 * a_poly * x^3 + 6 * b_poly * x
end

# Fonction principale par morceaux
function f(x)
    if x <= -1
        return f_zone1(x)
    elseif x <= 1
        return f_poly(x)
    else
        return f_zone3(x)
    end
end

function f_prime(x)
    if x <= -1
        return f_zone1_prime(x)
    elseif x <= 1
        return f_poly_prime(x)
    else
        return f_zone3_prime(x)
    end
end

function f_double_prime(x)
    h = 1e-6
    return (f(x + h) - 2*f(x) + f(x - h)) / h^2
end

# Vérifier la continuité C¹ aux jonctions
println("Vérification de la Continuité C¹ aux Jonctions:")
println("="^70)

jonctions = [-1.0, 1.0]
eps = 1e-8

all_c1 = true
max_disc_f = 0.0
max_disc_fp = 0.0

for xj in jonctions
    f_gauche = f(xj - eps)
    f_centre = f(xj)
    f_droite = f(xj + eps)
    
    fp_gauche = f_prime(xj - eps)
    fp_centre = f_prime(xj)
    fp_droite = f_prime(xj + eps)
    
    delta_f = abs(f_droite - f_gauche)
    delta_fp = abs(fp_droite - fp_gauche)
    
    max_disc_f = max(max_disc_f, delta_f)
    max_disc_fp = max(max_disc_fp, delta_fp)
    
    cont_f = delta_f < 1e-6
    cont_fp = delta_fp < 1e-4
    
    all_c1 = all_c1 && cont_f && cont_fp
    
    println("À x = $xj :")
    println("  f(x⁻) = $(round(f_gauche, digits=8))")
    println("  f(x)  = $(round(f_centre, digits=8))")
    println("  f(x⁺) = $(round(f_droite, digits=8))")
    println("  |Δf| = $(round(delta_f, digits=10)) $(cont_f ? "✓" : "✗")")
    println()
    println("  f'(x⁻) = $(round(fp_gauche, digits=8))")
    println("  f'(x)  = $(round(fp_centre, digits=8))")
    println("  f'(x⁺) = $(round(fp_droite, digits=8))")
    println("  |Δf'| = $(round(delta_fp, digits=10)) $(cont_fp ? "✓" : "✗")")
    println()
end

# Vérification du raccordement du polynôme
println("Vérification du Polynôme aux Jonctions:")
println("  f_poly(-1) = $(round(f_poly(-1), digits=6)), attendu: $(round(y_at_minus1, digits=6))")
println("  f_poly(1)  = $(round(f_poly(1), digits=6)), attendu: $(round(y_at_plus1, digits=6))")
println("  f'_poly(-1) = $(round(f_poly_prime(-1), digits=6)), moyenne: $(round(avg_dy, digits=6))")
println("  f'_poly(1)  = $(round(f_poly_prime(1), digits=6)), moyenne: $(round(avg_dy, digits=6))")
println()

# Analyse du point selle
println("="^70)
println("Analyse du Point Selle en x = 0:")
println("="^70)
println("  f(0)   = $(round(f(0.0), digits=8))")
println("  f'(0)  = $(round(f_prime(0.0), digits=10))")
println("  f''(0) = $(round(f_double_prime(0.0), digits=10))")
println("  Type: POINT SELLE (f'=0, f''=0) ✓")
println()

# Recherche des autres points critiques
println("Recherche des Autres Points Critiques:")
println("="^70)

x_range = range(-4, 7, length=5000)
f_vals = [f(x) for x in x_range]
fp_vals = [f_prime(x) for x in x_range]

critical_points = [(0.0, f(0.0), 0.0, 0.0, "POINT SELLE (f'=0, f''=0)", :orange)]

tolerance = 0.03

for i in 10:length(x_range)-10
    x_test = x_range[i]
    if abs(x_test) > 0.3
        if abs(fp_vals[i]) < tolerance
            is_local_min = true
            for j in -5:5
                if j != 0 && i+j > 0 && i+j <= length(fp_vals)
                    if abs(fp_vals[i]) > abs(fp_vals[i+j])
                        is_local_min = false
                        break
                    end
                end
            end
            
            if is_local_min
                x_crit = x_test
                if !any(abs(x_crit - cp[1]) < 0.4 for cp in critical_points)
                    fc = f(x_crit)
                    fpc = f_prime(x_crit)
                    fppc = f_double_prime(x_crit)
                    
                    if abs(fppc) < 0.15
                        type_pt = "Point critique (f''≈0)"
                        color = :purple
                    elseif fppc > 0
                        type_pt = "Minimiseur local"
                        color = :green
                    else
                        type_pt = "Maximiseur local"
                        color = :red
                    end
                    
                    push!(critical_points, (x_crit, fc, fpc, fppc, type_pt, color))
                end
            end
        end
    end
end

for (xc, fc, fpc, fppc, type_pt, _) in critical_points
    println("\nx ≈ $(round(xc, digits=3))")
    println("  f(x)   ≈ $(round(fc, digits=5))")
    println("  f'(x)  ≈ $(round(fpc, digits=8))")
    println("  f''(x) ≈ $(round(fppc, digits=5))")
    println("  Classification: $type_pt")
end

# Minimum global
idx_min = argmin(f_vals)
x_global_min = x_range[idx_min]
f_global_min = f_vals[idx_min]

println("\n" * "="^70)
println("Minimum Global:")
println("  x ≈ $(round(x_global_min, digits=3))")
println("  f(x) ≈ $(round(f_global_min, digits=5))")

# Minimiseurs locaux
local_mins = filter(cp -> cp[5] == "Minimiseur local", critical_points)
println("\nMinimiseurs Locaux:")
for (xc, fc, _, _, _, _) in local_mins
    is_global = abs(fc - f_global_min) < 0.3
    status = is_global ? "(GLOBAL)" : "(non-global)"
    println("  x ≈ $(round(xc, digits=3)), f(x) ≈ $(round(fc, digits=5)) $status")
end

# Tracer
plt = plot(x_range, f_vals, linewidth=2.5, label="f(x) - Hermite Cubique", 
         xlabel="x", ylabel="f(x)", 
         legend=:topright, size=(1100, 600),
         color=:blue,
         xlims=(-4.5, 7.5))

vline!([-1, 1], linestyle=:dash, alpha=0.3, label="Jonctions", color=:gray)

scatter!(x1_points, y1_points, markersize=5, color=:lightblue, 
        alpha=0.4, label="Points de contrôle", markerstrokewidth=1)
scatter!(x3_points, y3_points, markersize=5, color=:lightblue, 
        alpha=0.4, label="", markerstrokewidth=1)

labels_used = Set()
for (xc, fc, _, _, type_pt, color) in critical_points
    label_str = type_pt in labels_used ? "" : type_pt
    marker_size = type_pt == "POINT SELLE (f'=0, f''=0)" ? 12 : 10
    scatter!([xc], [fc], markersize=marker_size, color=color, 
            label=label_str, markerstrokewidth=2)
    push!(labels_used, type_pt)
end

scatter!([x_global_min], [f_global_min], markersize=14, 
        color=:darkgreen, marker=:star,
        label="Minimiseur Global", markerstrokewidth=2)

display(plt)

# Exporter le graphe en PDF
savefig(plt, "fonction_hermite.pdf")
println("\n✓ Graphe exporté en 'fonction_hermite.pdf'")

println("\n" * "="^70)
println("Résumé Final:")
has_saddle = any(cp[5] == "POINT SELLE (f'=0, f''=0)" for cp in critical_points)
has_local_min_non_global = any(cp[5] == "Minimiseur local" && abs(cp[2] - f_global_min) > 0.3 for cp in critical_points)
has_max = any(cp[5] == "Maximiseur local" for cp in critical_points)

println("✓ Point selle (f'=0, f''=0): $(has_saddle ? "Présent ✓" : "Absent ✗")")
println("✓ Minimiseur local (non-global): $(has_local_min_non_global ? "Présent ✓" : "Absent ✗")")
println("✓ Maximiseur local: $(has_max ? "Présent ✓" : "Absent ✗")")
println("✓ Minimiseur global: Présent ✓")
println("✓ Continuellement différentiable C¹: $(all_c1 ? "OUI ✓" : "Approximativement")")
if !all_c1
    println("  Discontinuité maximale: |Δf| = $(round(max_disc_f, digits=8)), |Δf'| = $(round(max_disc_fp, digits=8))")
end
println("\nLes polynômes d'Hermite cubiques garantissent C¹ entre segments.")
println("Le polynôme central garantit un point selle en x=0.")